In [ ]:
# 라이브러리 로딩
from selenium import webdriver as wb
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup as bs
import undetected_chromedriver as uc
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from time import sleep
from collections import defaultdict


In [ ]:
from selenium import webdriver as wb
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.action_chains import ActionChains
from webdriver_manager.chrome import ChromeDriverManager
from time import sleep
import json

subject_map = {
    '소프트웨어 설계': 1,
    '소프트웨어 개발': 2,
    '데이터베이스 구축': 3,
    '프로그래밍 언어 활용': 4,
    '정보시스템 구축관리': 5
}

driver = wb.Chrome(service=Service(ChromeDriverManager().install()))
driver.get('https://newbt.kr/')
sleep(3)

# 검색 및 시험지 설정 생략...

search = driver.find_element(By.ID, 'exam-search')
search.send_keys('정보처리기사')

# 정보처리기사 선택
pro_1 = driver.find_elements(By.CSS_SELECTOR,'body > div:nth-child(6) > div:nth-child(4) > div:nth-child(35) > div' )
pro_1[0].click()

#연도 선택하기
pro_1 = driver.find_elements(By.CSS_SELECTOR,'#serialSelect' )
pro_1[0].click()
sleep(1)

# 2022년 2회 클릭
pro_1 = driver.find_elements(By.CSS_SELECTOR,'#serialList > li:nth-child(1)' )
pro_1[0].click()

# 설정 버튼 클릭 (톱니바퀴)
pro_5 = driver.find_elements(By.CSS_SELECTOR, '#main > div > div.col-sm-8.blog-main > div.blog-post.question > label.setting')
pro_5[0].click()
sleep(1)

# 출력 방법 > 순차 보기 버튼 클릭
pro_5 = driver.find_elements(By.CSS_SELECTOR, '#settingModal > div > div > div.modal-body > div:nth-child(2) > div.custom-control.custom-radio.mb-3 > label')
pro_5[0].click()

# 설정하기 버튼 클릭
pro_5 = driver.find_elements(By.CSS_SELECTOR, '#settingModal > div > div > div.modal-footer > button.btn.btn-primary')
pro_5[0].click()
sleep(1)

# 처음 버튼 클릭
pro_5 = driver.find_elements(By.CSS_SELECTOR, '#main > div > div.col-sm-8.blog-main > div.mb-4 > a:nth-child(1)')
pro_5[0].click()
sleep(1)

result_list = []

for i in range(100):
    try:
        sleep(1)

        # 정답 보기 클릭
        try:
            driver.find_element(By.CSS_SELECTOR, '#main > div > div.col-sm-8.blog-main > div.mb-4 > a.btn.btn-sm.btn-secondary').click()
            sleep(1)
        except:
            print("정답 보기 버튼 없음")

        # 과목명 → 숫자
        subject_name = driver.find_element(By.CSS_SELECTOR, '#categorySelect').text.strip()
        subject_no = subject_map.get(subject_name, 0)

        # 문제 텍스트
        question_elem = driver.find_element(By.CSS_SELECTOR, '#main > div > div.col-sm-8.blog-main > div.blog-post.question > h5')
        question_text = question_elem.text.strip()[2:].strip()

        # 문제 상세설명 <pre> 태그 텍스트 추출
        try:
            pre_elem = driver.find_element(By.CSS_SELECTOR, '#main > div > div.col-sm-8.blog-main > div.blog-post.question > pre')
            question_detail = pre_elem.text.strip()
        except:
            question_detail = None

        # 보기 및 정답 찾기
        choices = []
        answer = None

        for j in range(1, 5):
            elem = driver.find_element(By.CSS_SELECTOR, f'#example{j}')
            full_text = elem.text.strip()
            cleaned_text = full_text[2:].strip()

            choices.append(cleaned_text)

            if '\n' in full_text:
                answer = j

        # 해설 보기 클릭 전 ↓ 키 5번 누르기
        actions = ActionChains(driver)
        for _ in range(5):
            actions.send_keys(Keys.ARROW_DOWN)
        actions.perform()
        sleep(1)

        # 해설 URL 추출
        try:
            exp_btn = driver.find_element(By.CSS_SELECTOR, '#main > div > div.col-sm-8.blog-main > div.resolveCover > div > div')
            
            exp_btn.click()
            sleep(0.5)
            href_elem = driver.find_element(By.CSS_SELECTOR, '#main > div > div.col-sm-8.blog-main > div.resolveCover > div > a')
            explanation_url = href_elem.get_attribute('href')
        except:
            explanation_url = None

        data = {
            "subject_no": subject_no,
            "question": question_text,
            "question_detail": question_detail,
            "choice_1": choices[0],
            "choice_2": choices[1],
            "choice_3": choices[2],
            "choice_4": choices[3],
            "answer": answer,
            "explanation": explanation_url
        }

        result_list.append(data)
        print(f"[{i+1}] 저장 완료: {subject_name}")

        # 다음 문제로 이동
        next_btn = driver.find_element(By.CSS_SELECTOR, '#main > div > div.col-sm-8.blog-main > div.mb-4 > a:nth-child(4)')
        next_btn.click()

    except Exception as e:
        print(f"[{i+1}] 오류 발생: {e}")
        driver.refresh()
        sleep(2)

# 저장
with open('info_exam_data.json', 'w', encoding='utf-8') as f:
    json.dump(result_list, f, ensure_ascii=False, indent=4)


In [61]:
from selenium import webdriver as wb
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from time import sleep
import json

# 과목명 → 번호 매핑
subject_map = {
    '소프트웨어 설계': 1,
    '소프트웨어 개발': 2,
    '데이터베이스 구축': 3,
    '프로그래밍 언어 활용': 4,
    '정보시스템 구축관리': 5
}

driver = wb.Chrome(service=Service(ChromeDriverManager().install()))
driver.get('https://newbt.kr/')
sleep(3)

# 검색 및 시험지 설정은 이전 코드와 동일하게 진행
# 입력창에 "정보처리기사" 입력하기
search = driver.find_element(By.ID, 'exam-search')
search.send_keys('정보처리기사')

# 정보처리기사 선택
pro_1 = driver.find_elements(By.CSS_SELECTOR,'body > div:nth-child(6) > div:nth-child(4) > div:nth-child(35) > div' )
pro_1[0].click()

#연도 선택하기
pro_1 = driver.find_elements(By.CSS_SELECTOR,'#serialSelect' )
pro_1[0].click()
sleep(1)

# 2022년 2회 클릭
pro_1 = driver.find_elements(By.CSS_SELECTOR,'#serialList > li:nth-child(1)' )
pro_1[0].click()

# 설정 버튼 클릭 (톱니바퀴)
pro_5 = driver.find_elements(By.CSS_SELECTOR, '#main > div > div.col-sm-8.blog-main > div.blog-post.question > label.setting')
pro_5[0].click()
sleep(1)

# 출력 방법 > 순차 보기 버튼 클릭
pro_5 = driver.find_elements(By.CSS_SELECTOR, '#settingModal > div > div > div.modal-body > div:nth-child(2) > div.custom-control.custom-radio.mb-3 > label')
pro_5[0].click()

# 설정하기 버튼 클릭
pro_5 = driver.find_elements(By.CSS_SELECTOR, '#settingModal > div > div > div.modal-footer > button.btn.btn-primary')
pro_5[0].click()
sleep(1)

# 처음 버튼 클릭
pro_5 = driver.find_elements(By.CSS_SELECTOR, '#main > div > div.col-sm-8.blog-main > div.mb-4 > a:nth-child(1)')
pro_5[0].click()
sleep(1)

# 결과 저장 리스트
result_list = []

for i in range(10):  # 100문제
    try:
        sleep(1)

        # 정답 보기 버튼 클릭
        try:
            driver.find_element(By.CSS_SELECTOR,'#main > div > div.col-sm-8.blog-main > div.mb-4 > a.btn.btn-sm.btn-secondary').click()
            sleep(1)
        except:
            print("정답 보기 버튼 클릭 실패")

        # 과목명 -> 숫자로 매핑
        subject_name = driver.find_element(By.CSS_SELECTOR, '#categorySelect').text.strip()
        subject_no = subject_map.get(subject_name, 0)

        # 문제 텍스트
        question_elem = driver.find_element(By.CSS_SELECTOR, '#main > div > div.col-sm-8.blog-main > div.blog-post.question > h5')
        question_text = question_elem.text.strip()[2:].strip()

        # 이미지가 포함된 경우 이미지 src 추출
        try:
            img_elem = driver.find_element(By.CSS_SELECTOR, '#main img')
            image_url = img_elem.get_attribute('src')
        except:
            image_url = None

        # 보기 추출 및 정답 판별
        choices = []
        answer = None

        for j in range(1, 5):
            elem = driver.find_element(By.CSS_SELECTOR, f'#example{j}')
            full_text = elem.text.strip()
            cleaned_text = full_text[2:].strip()

            choices.append(cleaned_text)

            # '\n' 포함된 항목이 정답
            if '\n' in full_text:
                answer = j

        # 해설 URL 추출
        try:
            exp_btn = driver.find_element(By.CSS_SELECTOR, '#main > div > div.col-sm-8.blog-main > div.resolveCover > div > div')
            exp_btn.click()
            sleep(0.5)
            href_elem = driver.find_element(By.CSS_SELECTOR, '#main > div > div.col-sm-8.blog-main > div.resolveCover > div > div > a')
            explanation_url = href_elem.get_attribute('href')
        except:
            explanation_url = None

        data = {
            "subject_no": subject_no,
            "question": question_text,
            "image_url": image_url,
            "choice_1": choices[0],
            "choice_2": choices[1],
            "choice_3": choices[2],
            "choice_4": choices[3],
            "answer": answer,
            "explanation": explanation_url
        }

        result_list.append(data)
        print(f"[{i+1}] {subject_name} 저장 완료")

        # 다음 문제로 이동
        next_btn = driver.find_element(By.CSS_SELECTOR, '#main > div > div.col-sm-8.blog-main > div.mb-4 > a:nth-child(4)')
        next_btn.click()

    except Exception as e:
        print(f"[{i+1}] 오류 발생: {e}")
        driver.refresh()
        sleep(2)

# 저장
with open('info_exam_data.json', 'w', encoding='utf-8') as f:
    json.dump(result_list, f, ensure_ascii=False, indent=4)


[1] 소프트웨어 설계 저장 완료
[2] 소프트웨어 설계 저장 완료
[3] 소프트웨어 설계 저장 완료
[4] 소프트웨어 설계 저장 완료
[5] 소프트웨어 설계 저장 완료
[6] 소프트웨어 설계 저장 완료
[7] 소프트웨어 설계 저장 완료
[8] 소프트웨어 설계 저장 완료
[9] 소프트웨어 설계 저장 완료
[10] 소프트웨어 설계 저장 완료


In [58]:
from selenium import webdriver as wb
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from time import sleep

driver = wb.Chrome(service=Service(ChromeDriverManager().install()))
driver.get('https://newbt.kr/')
sleep(3)

# 검색 및 시험지 설정 코드 (생략, 기존 코드와 동일)
# ...

# 입력창에 "정보처리기사" 입력하기
search = driver.find_element(By.ID, 'exam-search')
search.send_keys('정보처리기사')

# 정보처리기사 선택
pro_1 = driver.find_elements(By.CSS_SELECTOR,'body > div:nth-child(6) > div:nth-child(4) > div:nth-child(35) > div' )
pro_1[0].click()

#연도 선택하기
pro_1 = driver.find_elements(By.CSS_SELECTOR,'#serialSelect' )
pro_1[0].click()
sleep(1)

# 2022년 2회 클릭
pro_1 = driver.find_elements(By.CSS_SELECTOR,'#serialList > li:nth-child(1)' )
pro_1[0].click()

# 설정 버튼 클릭 (톱니바퀴)
pro_5 = driver.find_elements(By.CSS_SELECTOR, '#main > div > div.col-sm-8.blog-main > div.blog-post.question > label.setting')
pro_5[0].click()
sleep(1)

# 출력 방법 > 순차 보기 버튼 클릭
pro_5 = driver.find_elements(By.CSS_SELECTOR, '#settingModal > div > div > div.modal-body > div:nth-child(2) > div.custom-control.custom-radio.mb-3 > label')
pro_5[0].click()

# 설정하기 버튼 클릭
pro_5 = driver.find_elements(By.CSS_SELECTOR, '#settingModal > div > div > div.modal-footer > button.btn.btn-primary')
pro_5[0].click()
sleep(1)

# 처음 버튼 클릭
pro_5 = driver.find_elements(By.CSS_SELECTOR, '#main > div > div.col-sm-8.blog-main > div.mb-4 > a:nth-child(1)')
pro_5[0].click()
sleep(1)

# 결과를 저장할 리스트
result_list = []

#for i in range(100):  # 100문제 반복
    #try:
        #sleep(1)

        # 정답 보기 클릭
try:
    driver.find_element(By.CSS_SELECTOR,'#main > div > div.col-sm-8.blog-main > div.mb-4 > a.btn.btn-sm.btn-secondary').click()
    sleep(1)
except:
    print("정답 보기 버튼 없음 또는 클릭 실패")

In [ ]:
driver.find_element(By.CSS_SELECTOR,'body')

In [60]:
body = driver.find_element(By.CSS_SELECTOR,'body')
for i in range(0,10):
    body.send_keys(Keys.ARROW_DOWN)
sleep(1)

In [ ]:
#main > div > div.col-sm-8.blog-main > div.resolveCover > div > div

In [ ]:
# 해설 URL (해설 버튼 클릭 > 링크 추출)
explanation_url = None
try:
    explanation_btn = driver.find_element(By.CSS_SELECTOR, '#main > div > div.col-sm-8.blog-main > div.blog-post.question > div.mt-3 > a')
    explanation_url = explanation_btn.get_attribute('href')
except:
    explanation_url = "없음"

In [57]:
from selenium import webdriver as wb
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from time import sleep

driver = wb.Chrome(service=Service(ChromeDriverManager().install()))
driver.get('https://newbt.kr/')
sleep(3)

# 검색 및 시험지 설정 코드 (생략, 기존 코드와 동일)
# ...

# 입력창에 "정보처리기사" 입력하기
search = driver.find_element(By.ID, 'exam-search')
search.send_keys('정보처리기사')

# 정보처리기사 선택
pro_1 = driver.find_elements(By.CSS_SELECTOR,'body > div:nth-child(6) > div:nth-child(4) > div:nth-child(35) > div' )
pro_1[0].click()

#연도 선택하기
pro_1 = driver.find_elements(By.CSS_SELECTOR,'#serialSelect' )
pro_1[0].click()
sleep(1)

# 2022년 2회 클릭
pro_1 = driver.find_elements(By.CSS_SELECTOR,'#serialList > li:nth-child(1)' )
pro_1[0].click()

# 설정 버튼 클릭 (톱니바퀴)
pro_5 = driver.find_elements(By.CSS_SELECTOR, '#main > div > div.col-sm-8.blog-main > div.blog-post.question > label.setting')
pro_5[0].click()
sleep(1)

# 출력 방법 > 순차 보기 버튼 클릭
pro_5 = driver.find_elements(By.CSS_SELECTOR, '#settingModal > div > div > div.modal-body > div:nth-child(2) > div.custom-control.custom-radio.mb-3 > label')
pro_5[0].click()

# 설정하기 버튼 클릭
pro_5 = driver.find_elements(By.CSS_SELECTOR, '#settingModal > div > div > div.modal-footer > button.btn.btn-primary')
pro_5[0].click()
sleep(1)

# 처음 버튼 클릭
pro_5 = driver.find_elements(By.CSS_SELECTOR, '#main > div > div.col-sm-8.blog-main > div.mb-4 > a:nth-child(1)')
pro_5[0].click()
sleep(1)

# 결과를 저장할 리스트
result_list = []

for i in range(100):  # 100문제 반복
    try:
        sleep(1)

        # 정답 보기 클릭
        try:
            driver.find_element(By.CSS_SELECTOR,'#main > div > div.col-sm-8.blog-main > div.mb-4 > a.btn.btn-sm.btn-secondary').click()
            sleep(1)
        except:
            print("정답 보기 버튼 없음 또는 클릭 실패")

        # 과목명
        subject_no = driver.find_element(By.CSS_SELECTOR, '#categorySelect').text.strip()

        # 문제
        question_elem = driver.find_element(By.CSS_SELECTOR, '#main > div > div.col-sm-8.blog-main > div.blog-post.question > h5')
        question = question_elem.text.strip()[2:].strip()  # 문제번호 제거

        # 보기
        choices = []
        for j in range(1, 5):
            choice_text = driver.find_element(By.CSS_SELECTOR, f'#example{j}').text.strip()[2:].strip()
            choices.append(choice_text)

        # 정답 추출 (정답에 check 아이콘 있음)
        answer = None
        for j in range(1, 5):
            elem = driver.find_element(By.CSS_SELECTOR, f'#example{j}')
            if '정답입니다' in elem.text:
                answer = j
                break

        # 해설 URL (해설 버튼 클릭 > 링크 추출)
        explanation_url = None
        try:
            explanation_btn = driver.find_element(By.CSS_SELECTOR, '#main > div > div.col-sm-8.blog-main > div.blog-post.question > div.mt-3 > a')
            explanation_url = explanation_btn.get_attribute('href')
        except:
            explanation_url = "없음"

        # 결과 저장
        data = {
            "subject_no": subject_no,
            "question": question,
            "choice_1": choices[0],
            "choice_2": choices[1],
            "choice_3": choices[2],
            "choice_4": choices[3],
            "answer": answer,
            "explanation": explanation_url
        }

        result_list.append(data)
        print(f"[{i+1}] {subject_no} 문제 저장 완료")

        # 다음 문제로 이동
        next_btn = driver.find_element(By.CSS_SELECTOR, '#main > div > div.col-sm-8.blog-main > div.mb-4 > a:nth-child(4)')
        next_btn.click()

    except Exception as e:
        print(f"[{i+1}] 오류 발생: {e}")
        driver.refresh()
        sleep(2)

# 저장 확인
import json
with open('info_exam_data.json', 'w', encoding='utf-8') as f:
    json.dump(result_list, f, ensure_ascii=False, indent=4)


[1] 소프트웨어 설계 문제 저장 완료
[2] 소프트웨어 설계 문제 저장 완료
[3] 소프트웨어 설계 문제 저장 완료
[4] 소프트웨어 설계 문제 저장 완료
[5] 소프트웨어 설계 문제 저장 완료
[6] 소프트웨어 설계 문제 저장 완료
[7] 소프트웨어 설계 문제 저장 완료
[8] 소프트웨어 설계 문제 저장 완료
[9] 소프트웨어 설계 문제 저장 완료
[10] 소프트웨어 설계 문제 저장 완료
[11] 소프트웨어 설계 문제 저장 완료
[12] 소프트웨어 설계 문제 저장 완료
[13] 소프트웨어 설계 문제 저장 완료
[14] 소프트웨어 설계 문제 저장 완료
[15] 소프트웨어 설계 문제 저장 완료
[16] 소프트웨어 설계 문제 저장 완료
[17] 소프트웨어 설계 문제 저장 완료
[18] 소프트웨어 설계 문제 저장 완료
[19] 소프트웨어 설계 문제 저장 완료
[20] 소프트웨어 설계 문제 저장 완료
[21] 소프트웨어 개발 문제 저장 완료
[22] 소프트웨어 개발 문제 저장 완료
[23] 소프트웨어 개발 문제 저장 완료
[24] 소프트웨어 개발 문제 저장 완료
[25] 소프트웨어 개발 문제 저장 완료
[26] 소프트웨어 개발 문제 저장 완료
[27] 소프트웨어 개발 문제 저장 완료
[28] 소프트웨어 개발 문제 저장 완료
[29] 소프트웨어 개발 문제 저장 완료
[30] 소프트웨어 개발 문제 저장 완료
[31] 소프트웨어 개발 문제 저장 완료
[32] 소프트웨어 개발 문제 저장 완료
[33] 소프트웨어 개발 문제 저장 완료
[34] 소프트웨어 개발 문제 저장 완료
[35] 소프트웨어 개발 문제 저장 완료
[36] 소프트웨어 개발 문제 저장 완료
[37] 소프트웨어 개발 문제 저장 완료
[38] 소프트웨어 개발 문제 저장 완료
[39] 소프트웨어 개발 문제 저장 완료
[40] 데이터베이스 구축 문제 저장 완료
[41] 데이터베이스 구축 문제 저장 완료
[42] 데이터베이스 구축 문제 저장 완료
[43] 데이터베이스 구축 문제 저장 완료
[44] 데이터베이스 구축 문

In [56]:
from textwrap import dedent
# 라이브러리 로딩
from selenium import webdriver as wb
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup as bs
import undetected_chromedriver as uc
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from time import sleep
from collections import defaultdict

# OCR 기반 퀴즈 크롤링 코드 (Selenium + BeautifulSoup)
code = dedent(
    from selenium import webdriver
    from selenium.webdriver.common.by import By
    from selenium.webdriver.chrome.service import Service
    from selenium.webdriver.support.ui import WebDriverWait
    from selenium.webdriver.support import expected_conditions as EC
    from webdriver_manager.chrome import ChromeDriverManager
    from bs4 import BeautifulSoup
    from time import sleep

    # 과목명 → 번호 매핑
    subject_map = {
        '소프트웨어 설계': 1,
        '소프트웨어 개발': 2,
        '데이터베이스 구축': 3,
        '프로그래밍 언어 활용': 4,
        '정보시스템 구축관리': 5
    }

    # 드라이버 설정
    s = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=s)
    driver.get('https://newbt.kr/')
    sleep(2)

    # "정보처리기사" 입력
    search = driver.find_element(By.ID, 'exam-search')
    search.send_keys('정보처리기사')
    sleep(1)

    # 시험명 클릭
    driver.find_elements(By.CSS_SELECTOR, 'body > div:nth-child(6) > div:nth-child(4) > div:nth-child(35) > div')[0].click()
    sleep(1)

    # 연도 선택
    driver.find_elements(By.CSS_SELECTOR, '#serialSelect')[0].click()
    sleep(1)
    driver.find_elements(By.CSS_SELECTOR, '#serialList > li:nth-child(1)')[0].click()
    sleep(1)

    # 설정 (톱니바퀴 클릭 → 순차 보기 설정)
    driver.find_elements(By.CSS_SELECTOR, '#main > div > div.col-sm-8.blog-main > div.blog-post.question > label.setting')[0].click()
    sleep(1)
    driver.find_elements(By.CSS_SELECTOR, '#settingModal label')[0].click()  # 순차 보기
    driver.find_elements(By.CSS_SELECTOR, '#settingModal button.btn.btn-primary')[0].click()
    sleep(1)

    # 처음 문제로 이동
    driver.find_elements(By.CSS_SELECTOR, '#main > div > div.col-sm-8.blog-main > div.mb-4 > a:nth-child(1)')[0].click()
    sleep(1)

    quiz_list = []

    for i in range(100):  # 최대 100문제 반복
        # 정답 보기 버튼 클릭
        driver.find_elements(By.CSS_SELECTOR, '#main > div > div.col-sm-8.blog-main > div.mb-4 > a.btn.btn-sm.btn-secondary')[0].click()
        sleep(0.5)

        # 과목 추출 및 subject_no 매핑
        subject_text = driver.find_element(By.CSS_SELECTOR, '#categorySelect').text.strip()
        subject_no = subject_map.get(subject_text, 0)
        if subject_no == 0:
            print("일치하는 과목 없음, 다음 문제로")
            continue

        # 문제 추출
        question_text = driver.find_element(By.CSS_SELECTOR, '#main h5').text.strip()
        question_text = question_text[2:].strip()  # 번호 제거

        # 보기 추출 및 전처리
        choices_text = {}
        for j in range(1, 5):
            text = driver.find_element(By.CSS_SELECTOR, f'#example{j}').text.strip()
            choices_text[f'choice_{j}'] = text[2:].strip()  # '① 보기' → '보기'

        # 정답 추출
        answer_raw = driver.find_element(By.CSS_SELECTOR, '#answerDiv > div.alert.alert-primary.mt-2').text.strip()
        answer = int(answer_raw.replace("정답: ", "").strip())

        # 해설 추출
        explanation = driver.find_element(By.CSS_SELECTOR, '#answerDiv > div.card.mb-3 > div > div').text.strip()

        # 결과 저장
        quiz_data = {
            "subject_no": subject_no,
            "question": question_text,
            "choice_1": choices_text["choice_1"],
            "choice_2": choices_text["choice_2"],
            "choice_3": choices_text["choice_3"],
            "choice_4": choices_text["choice_4"],
            "answer": answer,
            "explanation": explanation
        }

        quiz_list.append(quiz_data)
        print(f"[{i+1}] 저장됨:", quiz_data["question"])

        # 다음 문제로 이동
        next_btn = driver.find_elements(By.CSS_SELECTOR, '#main > div > div.col-sm-8.blog-main > div.mb-4 > a.btn.btn-primary.float-end')
        if next_btn:
            next_btn[0].click()
            sleep(1)
        else:
            print("다음 문제 없음")
            break

    driver.quit()
    print(f"총 {len(quiz_list)}개 문제 수집 완료")
)


SyntaxError: invalid syntax (980957723.py, line 16)

In [ ]:
# 결과 저장
path = "/mnt/data/quiz_crawler.py"
with open(path, "w", encoding="utf-8") as f:
    f.write(code)

path

In [ ]:
# 1. 브라우저 접속
s = Service(ChromeDriverManager().install())
driver = wb.Chrome(service = s)
driver.get('https://newbt.kr/')
sleep(3)


# 입력창에 "정보처리기사" 입력하기
search = driver.find_element(By.ID, 'exam-search')
search.send_keys('정보처리기사')

# 정보처리기사 선택
pro_1 = driver.find_elements(By.CSS_SELECTOR,'body > div:nth-child(6) > div:nth-child(4) > div:nth-child(35) > div' )
pro_1[0].click()

#연도 선택하기
pro_1 = driver.find_elements(By.CSS_SELECTOR,'#serialSelect' )
pro_1[0].click()
sleep(1)

# 2022년 2회 클릭
pro_1 = driver.find_elements(By.CSS_SELECTOR,'#serialList > li:nth-child(1)' )
pro_1[0].click()

# 설정 버튼 클릭 (톱니바퀴)
pro_5 = driver.find_elements(By.CSS_SELECTOR, '#main > div > div.col-sm-8.blog-main > div.blog-post.question > label.setting')
pro_5[0].click()
sleep(1)

# 출력 방법 > 순차 보기 버튼 클릭
pro_5 = driver.find_elements(By.CSS_SELECTOR, '#settingModal > div > div > div.modal-body > div:nth-child(2) > div.custom-control.custom-radio.mb-3 > label')
pro_5[0].click()

# 설정하기 버튼 클릭
pro_5 = driver.find_elements(By.CSS_SELECTOR, '#settingModal > div > div > div.modal-footer > button.btn.btn-primary')
pro_5[0].click()
sleep(1)

# 처음 버튼 클릭
pro_5 = driver.find_elements(By.CSS_SELECTOR, '#main > div > div.col-sm-8.blog-main > div.mb-4 > a:nth-child(1)')
pro_5[0].click()
sleep(1)

# 100번 반복하는 for문 ( 

# 정답보기 버튼 클릭
pro_4 = driver.find_elements(By.CSS_SELECTOR,'#main > div > div.col-sm-8.blog-main > div.mb-4 > a.btn.btn-sm.btn-secondary')
pro_4[0].click()
sleep(1)


# 과목 분류
subject_no = driver.find_element(By.CSS_SELECTOR, '#categorySelect').text.strip()

if(subject_no=='소프트웨어 설계'):
    print(subject_no)
    question = driver.find_element(By.CSS_SELECTOR, '#main > div > div.col-sm-8.blog-main > div.blog-post.question > h5')
    print(question)

elif(subject_no=='소프트웨어 개발'):
    print(subject_no)
elif(subject_no=='데이터베이스 구축'):
    print(subject_no)
elif(subject_no=='프로그래밍 언어 활용'):
    print(subject_no)
elif(subject_no=='정보시스템 구축관리'):
    print(subject_no)
else:
    #바로 다음페이지로 이동 처리
    print("일치하는 과목 없음")
# 문제 데이터 담기
question = driver.find_element(By.CSS_SELECTOR, '#main > div > div.col-sm-8.blog-main > div.blog-post.question > h5')
question_re=question.text


# 데이터 전처리, (문제번호삭제)
question_re = question_re.replace(question_re[:2],'')


choices = {}
choices_text = {}

for i in range(1, 5):
    
    choices[f'choice{i}'] = driver.find_element(By.CSS_SELECTOR, f'#example{i}').text.strip()

    #전처리 하기 전 문자열에 "\n"가 포함되면 answer에 int형으로 저장
    
        
    text = choices[f'choice{i}']
    choices_text[f'choice{i}_re'] = text.replace(text[:2],'').strip()

    print(choices_text[f'choice{i}_re'])

    # 다운키 10번 해주기
body = driver.find_element(By.CSS_SELECTOR,'body')
for i in range(0,10):
    body.send_keys(Keys.ARROW_DOWN)
sleep(1)
    
# 해설보여주기 버튼 클릭
pro_5 = driver.find_elements(By.CSS_SELECTOR,'#main > div > div.col-sm-8.blog-main > div.resolveCover > div > div')
pro_5[0].click()




# ) 100번 반복하는 for문  

# 결과적으로 나는 DB에 담을 subject_no, question, choice_1,choice_2,choice_3,choice_4, answer, explanation 가 필요하다




In [ ]:
if '\n' in choice2:
    